# Tworzenie zasobów

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1: Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [42]:
import urllib.request
from bs4 import BeautifulSoup
import re

link = 'http://dobre-ksiazki.com.pl/'

def getTextfromURL(link):

    with urllib.request.urlopen(link) as response:
       html_doc = response.read()

    soup = BeautifulSoup(html_doc, 'html.parser')
    paragraphs = soup.find_all('p')

    text = ""
    for para in paragraphs:
        text += para.get_text()
        
    return text

def getLinksfromURL(link):
    links = []
    i = 0
    REGEX = r'http*'
    with urllib.request.urlopen(link) as response:
       html_doc = response.read()

    soup = BeautifulSoup(html_doc, 'html.parser')
    
    for link in soup.find_all('a'):
        if  re.search(REGEX, link.get('href')):
            links.append(link.get('href'))
            i += 1
        if i==5:
            break
        
    return links

def parser(link_org):
    text = getTextfromURL(link_org)
    links = getLinksfromURL(link_org)
    for link in links:
        text += getTextfromURL(link)
    return text


In [45]:
text = parser(link)

In [46]:
text

'\n\n\n\n120 PRZEPISÓW NA CZAS DETOKSU I PO DETOKSIEJaglany Detoks\npomógł odzyskać zdrowie tysiącom osób! Roślinny skalpel jest stale\nw natarciu! Każdego dnia post skutecznie pokonuje coraz więcej\nchorób cywilizacyjnych otyłość, cukrzycę, alergie pokarmowe,\ninfekcje!W kolejnej części...\n więcej\n\n\n\nTrzecia książka autorek bestsellerowych poradników\nkucharskich z przepisami dla najmłodszych smakoszy opartymi na\nmetodzie BLW – tym razem dla nieco starszych dzieci!Twoje dziecko\npotrafi już chodzić i głośno domagać się dokładki? Albo może wręcz\nprzeciwnie –protestuje przeciwko...\n więcej\n\n\n\nTrzynaście lat temu katastrofalna pandemia, nazywana\napokalipsą, zabiła miliardy.Dla tych, którzy przetrwali, to szansa\nna stworzenie nowego świata. Jednak apokalipsa nie była zwyczajnym\nwirusem. U części ocalałych obudziła dziwne moce potrafią\nuzdrawiać, wyrządzać szkody, a nawet...\n więcej\n\n\n\nGratka dla posiadaczy niebieskiej wersji Map!Dodatek do\npodstawowego wydania książk

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników).


In [54]:
def generate_conll(text = "Kate uses IPhone 5 and IPad. Kate bought 2 lolipops."):
    import nltk
    import string
    out = []
    sent_text = nltk.sent_tokenize(text)
    for sentence in sent_text:
        tokenized_text = nltk.word_tokenize(sentence)
        #print(tokenized_text)
        tags = nltk.pos_tag(tokenized_text)
        #print(tags)
        num = 1
        for word, tag in zip(tokenized_text, tags):
            out.append([num, word, tag[1], int(word.isdigit()), int(word in string.punctuation)])
            num +=1
    return out

conll = generate_conll()
conll


[[1, 'Kate', 'NNP', 0, 0],
 [2, 'uses', 'VBZ', 0, 0],
 [3, 'IPhone', 'NNP', 0, 0],
 [4, '5', 'CD', 1, 0],
 [5, 'and', 'CC', 0, 0],
 [6, 'IPad', 'NNP', 0, 0],
 [7, '.', '.', 0, 1],
 [1, 'Kate', 'NNP', 0, 0],
 [2, 'bought', 'VBD', 0, 0],
 [3, '2', 'CD', 1, 0],
 [4, 'lolipops', 'NNS', 0, 0],
 [5, '.', '.', 0, 1]]


Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

**Zadanie**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [26]:
import nltk
import string

def iob_tag(text, markers):
    markers_txt = []
    for marker in markers:
        markers_txt.append(nltk.word_tokenize(text[marker["begin"]:marker["end"]]))
    basic_tags = []
    for marker in markers_txt:
        basic_tags.append((marker[0], 'B'))
        if len(marker) > 1:
            for i in range(1,len(marker)):
                basic_tags.append((marker[i], 'I'))
    text_tokens = nltk.word_tokenize(text)
    out = []
    iterator = 0
    breaker = len(basic_tags)
    for token in text_tokens:
        
        if breaker>iterator and token == basic_tags[iterator][0]:
            #print(token)
            out.append(basic_tags[iterator])
            iterator += 1
        else:
            out.append((token,'O'))   
    return out

def generate_CONLL(text = "Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.", devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}]):
    out = []
    sent_text = nltk.sent_tokenize(text)
    iob_tags = iob_tag(text,devices)
    for sentence in sent_text:
        tokenized_text = nltk.word_tokenize(sentence)
        tags = nltk.pos_tag(tokenized_text)
        num = 1
        for word, tag, tag_iob in zip(tokenized_text, tags, iob_tags):
            out.append([num, word, tag[1], int(word.isdigit()), int(word in string.punctuation)])
            num +=1
    for record, tag in zip(out, iob_tags):
        record.append(tag[1])
    return out

conll = generate_CONLL()
conll

[[1, 'Kate', 'NNP', 0, 0, 'O'],
 [2, 'uses', 'VBZ', 0, 0, 'O'],
 [3, 'IPhone', 'NNP', 0, 0, 'B'],
 [4, '5', 'CD', 1, 0, 'I'],
 [5, 'and', 'CC', 0, 0, 'O'],
 [6, 'IPad', 'NNP', 0, 0, 'B'],
 [7, '.', '.', 0, 1, 'O'],
 [1, 'Kate', 'NNP', 0, 0, 'O'],
 [2, 'bought', 'VBD', 0, 0, 'O'],
 [3, '2', 'CD', 1, 0, 'O'],
 [4, 'lolipops', 'NNS', 0, 0, 'O'],
 [5, '.', '.', 0, 1, 'O']]

In [56]:
"elo" in ["elo", "melo"]

True

In [57]:
"elo" == "elo"

True

In [21]:
cos = iob_tag("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.",[{"begin": 10, "end":18}, {"begin": 23, "end": 27}])

IPhone
5
IPad


In [22]:
cos

[('Kate', 'O'),
 ('uses', 'O'),
 ('IPhone', 'B'),
 ('5', 'I'),
 ('and', 'O'),
 ('IPad', 'B'),
 ('.', 'O'),
 ('Kate', 'O'),
 ('bought', 'O'),
 ('2', 'O'),
 ('lolipops', 'O'),
 ('.', 'O')]